# Udacity : Wrangle and Analyse Datas with WeRateADog

## Introduction

As part of the Data Analyst NanoDegree (D.A.N.D), the project is here to Wrangle datas and Analyze those datas coming from Tweets of the account WeRateDogs.
    So, will be using Twitter datas, connexion to the API to control datas, and informations coming from Picture Analysis.

## What for this analysis ?

The analysis will be perfom in order to raise more interest about the datas. Twitter informations are poorly presented, but perhaps will we be able to render more informations about all this datas when compiled together.
We should exect some graphic analysis and tendance.

---

## Datas

### - Twitter Archive

An archive of more than 5K tweets is given as entry datas. Those are all original tweets, replys and retweets from the WeRateDogs account for a period of time.

### - Twitter Image Prediction

This is a CSV of Image Prediction depicting Dogs. This CSV is to download from a specific URI ```https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv``` 

### - Twitter API

In order to grab additional datas, I'll be using the Twitter API. This will allow me to complete, validate all datas that were given through the Twitter Archive.

---

In [1]:
import pandas as pd
import numpy as np
import requests
import os
import tweepy
from tweepy import OAuthHandler
import json
import time
import datetime
import random
import os
import logging

import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

import seaborn as sns

## Gathering Data

As requested, we will retrieve first each piece of data into ```3 differents DataFrame variables``` before trying to analyse, merge, aso. By the way, image prediction will be retrieved by using the ```URI given``` and the ```Request``` library.

In [2]:
# the Twitter-Archive
twitterArchive = pd.read_csv('twitter-archive-enhanced-2.csv')
twitterArchive.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


#### Gathering data from URI for Image Prediction
Goal was here to use the `Request` library to retrieve data from an URI. By the way, because it is a TSV we are downloading, the `read_csv` had to precise the new separator which was a tabulation instead of default comma.

In [3]:
# Retrieving the Image Prediction
urlImagePrediction = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv';

data = requests.get(urlImagePrediction)

with open('image-prediction.tsv', mode='wb') as file:
    file.write(data.content)
    
imagePrediction = pd.read_csv('image-prediction.tsv', sep='\t')
imagePrediction.head(2)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True


#### Gathering data using the Tweety API
This was first time I used an API Tweety with Python, and I went rather well.
Used OAuth Authentication and then had to loop through results to gather new data selected to enhance the Twitter Archive

In [4]:
# Dealing with the Twitter API protected with OAuth authentication
consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

twitterApi = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True )

In [37]:
errors = []
datas = []

for tweetId in twitterArchive['tweet_id']:
    try:
        tweet = twitterApi.get_status(tweetId, tweet_mode='extended')._json 
        favCount = tweet['favorite_count'] # to see how many favorite the tweets had
        retweetCount = tweet['retweet_count'] # for counting the retweets
        userFollowersCount = tweet['user']['followers_count'] # to see how many followers the user had
        userFavCount = tweet['user']['favourites_count'] # to see how many favorites the user had
        creationDate = tweet['created_at'] # to see the datetime of tweet
        
        datas.append({'tweet_id': int(tweetId),
                      'nb_favorites': int(favCount),
                      'nb_retweets': int(retweetCount),
                      'nb_user_followers': int(userFollowersCount),
                      'nb_user_favourites': int(userFavCount),
                      'creation_date': pd.to_datetime(creationDate)})
    except Exception as e:
        logging.error(f'Tweet ID : {tweetId}, Error Message : {e}')
        errors.append(tweetId)

ERROR:root:Tweet ID : 888202515573088257, Error Message : [{'code': 144, 'message': 'No status found with that ID.'}]
ERROR:root:Tweet ID : 873697596434513921, Error Message : [{'code': 144, 'message': 'No status found with that ID.'}]
ERROR:root:Tweet ID : 872668790621863937, Error Message : [{'code': 144, 'message': 'No status found with that ID.'}]
ERROR:root:Tweet ID : 872261713294495745, Error Message : [{'code': 144, 'message': 'No status found with that ID.'}]
ERROR:root:Tweet ID : 869988702071779329, Error Message : [{'code': 144, 'message': 'No status found with that ID.'}]
ERROR:root:Tweet ID : 866816280283807744, Error Message : [{'code': 144, 'message': 'No status found with that ID.'}]
ERROR:root:Tweet ID : 861769973181624320, Error Message : [{'code': 144, 'message': 'No status found with that ID.'}]
ERROR:root:Tweet ID : 856602993587888130, Error Message : [{'code': 144, 'message': 'No status found with that ID.'}]
ERROR:root:Tweet ID : 851953902622658560, Error Message 

In order to be used later (and reloaded from a CSV file instead of a new series of call to the API, data was saved in a CSV file then relaoded to verify the process)

In [38]:
jsonTweets = pd.DataFrame(datas, columns=[
    'tweet_id', 
    'nb_favorites', 
    'nb_retweets', 
    'nb_user_followers', 
    'nb_user_favourites', 
    'creation_date'
])

# As it took more than 30 minutes to retrieve all datas from the Twitter API, 
# best is to keep a copy into a CSV file for later use
jsonTweets.to_csv('twitter_json.csv', encoding = 'utf-8', index=False)

In [5]:
# For later use, it will be possible to load this file
jsonTweets = pd.read_csv('twitter_json.csv', encoding = 'utf-8')
jsonTweets.head(5)

,tweet_id,nb_favorites,nb_retweets,nb_user_followers,nb_user_favourites,creation_date
0,892420643555336193,36289,7722,8774885,146020,2017-08-01 16:23:56+00:00
1,892177421306343426,31291,5714,8774885,146020,2017-08-01 00:17:27+00:00
2,891815181378084864,23565,3785,8774885,146020,2017-07-31 00:18:03+00:00
3,891689557279858688,39588,7884,8774885,146020,2017-07-30 15:58:51+00:00
4,891327558926688256,37806,8499,8774885,146020,2017-07-29 16:00:24+00:00


---

## Assessing Data

Two types assessment will be use :
+ Visual assessment: each piece of gathered data is displayed in the Jupyter Notebook for visual assessment purposes. Once displayed, data can additionally be assessed in an external application (e.g. Excel, text editor).
+ Programmatic assessment: pandas' functions and/or methods are used to assess the data.

> At least eight (8) data quality issues and two (2) tidiness issues are detected, and include the issues to clean to satisfy the Project Motivation. Each issue is documented in one to a few sentences each.

In [6]:
twitterArchive.info()
twitterArchive.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None


In [7]:
imagePrediction.info()
imagePrediction.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True


In [8]:
jsonTweets.info()
jsonTweets.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 6 columns):
tweet_id              2331 non-null int64
nb_favorites          2331 non-null int64
nb_retweets           2331 non-null int64
nb_user_followers     2331 non-null int64
nb_user_favourites    2331 non-null int64
creation_date         2331 non-null object
dtypes: int64(5), object(1)
memory usage: 109.3+ KB


,tweet_id,nb_favorites,nb_retweets,nb_user_followers,nb_user_favourites,creation_date
0,892420643555336193,36289,7722,8774885,146020,2017-08-01 16:23:56+00:00
1,892177421306343426,31291,5714,8774885,146020,2017-08-01 00:17:27+00:00
2,891815181378084864,23565,3785,8774885,146020,2017-07-31 00:18:03+00:00


In [9]:
twitterArchive['name'].value_counts()

None         745
a             55
Charlie       12
Oliver        11
Cooper        11
Lucy          11
Penny         10
Tucker        10
Lola          10
Bo             9
Winston        9
the            8
Sadie          8
Daisy          7
Toby           7
an             7
Bailey         7
Buddy          7
Oscar          6
Koda           6
Jack           6
Milo           6
Scout          6
Bella          6
Leo            6
Jax            6
Stanley        6
Rusty          6
Dave           6
George         5
            ... 
Rinna          1
Juckson        1
Chuq           1
Enchilada      1
Bonaparte      1
Dewey          1
Cheryl         1
Arlen          1
Swagger        1
Dale           1
Bloo           1
Severus        1
Pinot          1
Sprinkles      1
Leonidas       1
Shikha         1
Nugget         1
Sephie         1
Dudley         1
Jazzy          1
Tommy          1
Dutch          1
Saydee         1
Schnitzel      1
Tug            1
Arnold         1
Sailor         1
Billy         

### Data quality issues

+ Twitter Archive :
    + 2 Columns may need a type change ```retweeted_status_timestamp``` and ```timestamp``` to be dealt as ```datetime```
    + Archive seems to miss 69 ```expanded_urls```
    + Column ```tweet_id``` should be treated as ```object``` (aka ```str```) as it operates as an identifier and not a numeric value we will calculate with
    + Column ```name``` may contain bad datas like ```a```, ```the```. Dog name seems to all begins by an Uppercase. We will drop rows not containing correct name.
    + As expressed in the Project Detail, we should focus on original tweets and not on retweets nor reply. We will have to tidy this behaviour from the dataset

+ Image Prediction :
    + Once again, the ```tweet_id``` should be considered as an identifier, therefore, we should change its type to ```str``` (or ```object```)
    + Columns abbreviations could be hard to understand, we will rewrite all P# as prediction_# (ex. P1 => prediction_1)
    + Prediction has 3 different results, we will summarize it with only one ```global``` prediction which will be the prediction depicting a dog and with the higher confidence.

+ Json Tweets : 
    + Same remark considering the ```tweet_id``` column which will be updated as an ```object``` column
    + The ```creation_date``` column will change from ```object``` to ```datetime``` type

### Tidyness

+ Twitter Archive :
    + The 4 columns ```doggo```, ```floofer```, ```pupper```, ```puppo``` may need to be combine in a ```stage``` column

    + All dataset contains a ```tweet_id``` column which could be used as a join point in order to work with a single dataset.

## Cleaning Data

We will now clean datas regarding all the observations made before

In [10]:
# Making a copy of all 3 datasets
jsonTweetsCleaned = jsonTweets.copy()
twitterArchiveCleaned = twitterArchive.copy()
imagePredictionCleaned = imagePrediction.copy()

In [11]:
twitterArchiveCleaned.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

#### Data Cleaning 1 : 
Altering type of data for `tweet_id` column from `integer` to `string`

In [12]:
# Change datatype for ```tweet_id``` column for twitterArchive
twitterArchiveCleaned['tweet_id'] = twitterArchiveCleaned['tweet_id'].astype('str')

#### Data Cleaning 2 :
Converting both column `timestamp` and `retweeted_status_timestamp` from `string` to `datetime` objects

In [13]:
# Change datatype for `timestamp` columns
twitterArchiveCleaned['timestamp'] = pd.to_datetime(twitterArchiveCleaned['timestamp'])
twitterArchiveCleaned['retweeted_status_timestamp'] = pd.to_datetime(twitterArchiveCleaned['retweeted_status_timestamp'])

#### Data Cleaning 3 :
Removing from dataset all rows whose column `retweeted_status_id` is not null (meaning that the tweet is not original but retweet)

In [14]:
# Analyzing only original tweets with photo and not retweets
nbRetweets = sum(twitterArchiveCleaned.retweeted_status_id.notnull())
print(f'We are going to remove {nbRetweets} retweets from the twitterArchive')

We are going to remove 181 retweets from the twitterArchive


In [15]:
twitterArchiveCleaned = twitterArchiveCleaned[twitterArchiveCleaned.retweeted_status_id.isna()]


#### Data Cleaning 4 :
As we just have done with retweets, we are now looking for replies and going to remove all those not-original tweets from the dataset

In [16]:
# Dropping replies
nbReplies = sum(twitterArchiveCleaned.in_reply_to_status_id.notnull())
print(f'We are going to remove {nbReplies} replies from the twitterArchive')

We are going to remove 78 replies from the twitterArchive


In [17]:
twitterArchiveCleaned = twitterArchiveCleaned[twitterArchiveCleaned.in_reply_to_status_id.isna()]

#### Data Cleaning 5 : 
Removing not anymore necessary columns from the dataset `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`

In [18]:
twitterArchiveCleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2097 non-null object
in_reply_to_status_id         0 non-null float64
in_reply_to_user_id           0 non-null float64
timestamp                     2097 non-null datetime64[ns, UTC]
source                        2097 non-null object
text                          2097 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null datetime64[ns, UTC]
expanded_urls                 2094 non-null object
rating_numerator              2097 non-null int64
rating_denominator            2097 non-null int64
name                          2097 non-null object
doggo                         2097 non-null object
floofer                       2097 non-null object
pupper                        2097 non-null object
puppo                         2097 non-null object
dtypes: datet

In [19]:
# Dropping all columns with no more data
twitterArchiveCleaned.drop(columns={'retweeted_status_id', 'retweeted_status_timestamp', 'retweeted_status_user_id', 'in_reply_to_status_id', 'in_reply_to_user_id'}, inplace=True)

#### Data Cleaning 6 :
We are droping all rows whose `name` does not begin by an Uppercase as we noticed before that all dog name seems to begin by an uppercase character and that all name value begining by a lowercase meaned that it was a buggy name (article `a`, `the`, ...)

In [20]:
# Cleaning dataset from bad named dogs
# Dropping every dogname not beginning by an Uppercase
twitterArchiveCleaned.drop(
    twitterArchiveCleaned[twitterArchiveCleaned['name'].str.match(r'^[a-z].*')==True].index, 
    inplace=True
)

#### Data Cleaning 7 :
We are simplifying the source column by extracting the "text" content from the `<a href="uri"...></a>` using a regexp extraction

In [23]:
# Simplifying the source column by maintaining only text information between A brackets
twitterArchiveCleaned['source'] = twitterArchiveCleaned['source'].str.extract('^<a.*>(.*)</a>$')

In [24]:
twitterArchiveCleaned.source.value_counts()

Series([], Name: source, dtype: int64)

#### Data Cleaning 8 : 
Let's filter out all `expanded_urls` which are null values from the dataset

In [25]:
# Dropping tweets with missing data in expanded_urls
twitterArchiveCleaned = twitterArchiveCleaned[twitterArchiveCleaned.expanded_urls.notnull()]

In [26]:
twitterArchiveCleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1990 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id              1990 non-null object
timestamp             1990 non-null datetime64[ns, UTC]
source                0 non-null object
text                  1990 non-null object
expanded_urls         1990 non-null object
rating_numerator      1990 non-null int64
rating_denominator    1990 non-null int64
name                  1990 non-null object
doggo                 1990 non-null object
floofer               1990 non-null object
pupper                1990 non-null object
puppo                 1990 non-null object
dtypes: datetime64[ns, UTC](1), int64(2), object(9)
memory usage: 202.1+ KB


#### Data Cleaning 9 :
We are regrouping all the dog stage into a single column `stage`.

We first count all those values to check it there is no row with more than 1 stage.

Then we set the new column either to one of the stage or to none if no stage was set

In [27]:
twitterArchiveCleaned[['doggo', 'floofer', 'pupper', 'puppo']].describe()

,doggo,floofer,pupper,puppo
count,1990,1990,1990,1990
unique,2,2,2,2
top,None,None,None,None
freq,1910,1980,1773,1966


In [28]:
# Let's check if how many "stage" are set and if we do not have "conflicts"
dummy = lambda x: 0 if x == 'None' else 1

stages = ['doggo', 'floofer', 'pupper', 'puppo']

for stage in stages:
    twitterArchiveCleaned[stage] = twitterArchiveCleaned[stage].apply(dummy)

twitterArchiveCleaned['stage'] = twitterArchiveCleaned['doggo'] + \
                                 twitterArchiveCleaned['floofer'] + \
                                 twitterArchiveCleaned['pupper'] + \
                                 twitterArchiveCleaned['puppo']

twitterArchiveCleaned['stage'].value_counts()

0    1669
1     311
2      10
Name: stage, dtype: int64

It seems that we have 10 rows with 2 different stages. By default, we'll select the most numerous group first, and so on.

In [29]:
# Regrouping columns as ```dog_stage```
stages = ['floofer', 'puppo', 'doggo', 'pupper', 'none']

stage_none = lambda x: 1 if x == 0 else 0

twitterArchiveCleaned['none'] = twitterArchiveCleaned['stage'].apply(stage_none)

def regroupStages(df):
    stages = ['floofer', 'puppo', 'doggo', 'pupper', 'none']
    for stage in stages: 
        if (df[stage]==1):
            return stage;
    return stages[4]

twitterArchiveCleaned['stage'] = twitterArchiveCleaned.apply(lambda x: regroupStages(x), axis=1)

# Dropping column stages
twitterArchiveCleaned.drop(columns=stages, inplace=True)

## Source of the algo : https://stackoverflow.com/questions/54910931/add-a-new-column-to-python-dataframe-with-switch-case-like-of-regex


In [30]:
twitterArchiveCleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1990 entries, 0 to 2355
Data columns (total 9 columns):
tweet_id              1990 non-null object
timestamp             1990 non-null datetime64[ns, UTC]
source                0 non-null object
text                  1990 non-null object
expanded_urls         1990 non-null object
rating_numerator      1990 non-null int64
rating_denominator    1990 non-null int64
name                  1990 non-null object
stage                 1990 non-null object
dtypes: datetime64[ns, UTC](1), int64(2), object(6)
memory usage: 155.5+ KB


#### Data Cleaning 10 : 
We are cleaning here the rating numerator and denominator. There are a lot of different values, and numerator and denominator goes repectively from 0 to 1776 and from 0 to 150.

Considering distribution, it is choosen to stay with only denominator equal to 10 and numerator between 1 to 14 inclusive.

In [31]:
# Understanding and cleaning ratings
twitterArchiveCleaned[['rating_numerator', 'rating_denominator']].describe()

,rating_numerator,rating_denominator
count,1990.000000,1990.000000
mean,12.196482,10.376382
std,41.190615,5.735457
min,0.000000,7.000000
25%,10.000000,10.000000
50%,11.000000,10.000000
75%,12.000000,10.000000
max,1776.000000,150.000000


It seems we have here some datas off grid `1776` for max rating_numerator, `150` for max rating denominator and both have `0` as minimum (0/0 ???). We must clean this part too.

In [32]:
twitterArchiveCleaned.rating_denominator.value_counts().sort_index(ascending = False)

150       1
120       1
110       1
90        1
80        2
70        1
50        2
40        1
20        1
11        2
10     1976
7         1
Name: rating_denominator, dtype: int64

In [33]:
twitterArchiveCleaned.rating_numerator.value_counts().sort_index(ascending = False)

1776      1
420       1
165       1
144       1
121       1
99        1
88        1
84        1
80        1
75        1
50        1
45        1
44        1
27        1
26        1
24        1
14       34
13      280
12      470
11      397
10      411
9       139
8        91
7        46
6        30
5        33
4        13
3        18
2         7
1         4
0         1
Name: rating_numerator, dtype: int64

We have 2 possibilities here : 
* Either we ignore denominator divergent values ( <> 10 for denominator, 0< n <= 14 for nominator)
* Or we try to fix those behaviour the best we can

Let's do proposition nr 1 just because the loss of content will not be significant regarding the remaining tweets.

In [34]:
# So removing numerator == 0 and greater than 15 && removing denominator different than 10
twitterArchiveCleaned = twitterArchiveCleaned[twitterArchiveCleaned['rating_numerator'] < 15]
twitterArchiveCleaned = twitterArchiveCleaned[twitterArchiveCleaned['rating_numerator'] > 0]
twitterArchiveCleaned = twitterArchiveCleaned[twitterArchiveCleaned['rating_denominator'] == 10]

In [35]:
twitterArchiveCleaned[['rating_numerator', 'rating_denominator']].describe()

,rating_numerator,rating_denominator
count,1970.000000,1970.0
mean,10.651269,10.0
std,2.114477,0.0
min,1.000000,10.0
25%,10.000000,10.0
50%,11.000000,10.0
75%,12.000000,10.0
max,14.000000,10.0


In [36]:
# As we have only rating_denominator left, let's remove this no more necessary column
twitterArchiveCleaned.drop(columns='rating_denominator', inplace=True)
# ...and rename the nominator column as rating
twitterArchiveCleaned.rename(columns={'rating_numerator':'rating'}, inplace=True)

In [37]:
# Control
twitterArchiveCleaned.sample(5)

,tweet_id,timestamp,source,text,expanded_urls,rating,name,stage
2242,667911425562669056,2015-11-21 03:44:27+00:00,NaN,Wow. Armored dog here. Ready for battle. Face ...,https://twitter.com/dog_rates/status/667911425...,5,None,none
2284,667192066997374976,2015-11-19 04:05:59+00:00,NaN,*takes several long deep breaths* omg omg oMG ...,https://twitter.com/dog_rates/status/667192066...,12,None,none
1846,675822767435051008,2015-12-12 23:41:18+00:00,NaN,🎶 HELLO FROM THE OTHER SIIIIIIIIDE 🎶 10/10s ht...,https://twitter.com/dog_rates/status/675822767...,10,None,none
23,887473957103951883,2017-07-19 00:47:34+00:00,NaN,This is Canela. She attempted some fancy porch...,https://twitter.com/dog_rates/status/887473957...,13,Canela,none
2165,669367896104181761,2015-11-25 04:11:57+00:00,NaN,This is Chip. Chip's pretending to be choked. ...,https://twitter.com/dog_rates/status/669367896...,10,Chip,none


### Cleaning the Image Prediction

#### Data Cleaning 11 : 
Converting the `tweet_id` column from `integer` to `string` as it act as a reference

In [38]:
# Change datatype for ```tweet_id``` column for imagePrediction
imagePredictionCleaned['tweet_id'] = imagePredictionCleaned['tweet_id'].astype('str')

#### Data Cleaning 12 : 
Renaming columns `p1*`, `p2*`, `p3*` to more comprehensible column name

In [39]:
# Renaming columns p1*, p2*, p3* as prediction_#..
imagePredictionCleaned.rename(columns={'p1':'prediction_1', 'p2':'prediction_2', 'p3':'prediction_3', 'p1_conf':'prediction_1_conf', 'p2_conf':'prediction_2_conf', 'p3_conf':'prediction_3_conf', 'p1_dog':'prediction_1_dog', 'p2_dog':'prediction_2_dog', 'p3_dog':'prediction_3_dog'}, inplace=True)

### Cleaning the Json Tweets

#### Data Cleaning 13 :
Converting the `tweet_id` column from integer to string as it act as a reference

In [40]:
# Change datatype for ```tweet_id``` column for jsonTweets
jsonTweetsCleaned['tweet_id'] = jsonTweetsCleaned['tweet_id'].astype('str')

#### Data Cleaning 14 :
Converting the `creation_date` from `string` to `datetime`

In [41]:
# Change datatype for ```creation_date``` column for jsonTweets
jsonTweetsCleaned['creation_date'] = pd.to_datetime(jsonTweetsCleaned['creation_date'])

Let's check the resulting datasets

In [42]:
# Control jsonTweets
jsonTweetsCleaned.info()
jsonTweetsCleaned.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 6 columns):
tweet_id              2331 non-null object
nb_favorites          2331 non-null int64
nb_retweets           2331 non-null int64
nb_user_followers     2331 non-null int64
nb_user_favourites    2331 non-null int64
creation_date         2331 non-null datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), int64(4), object(1)
memory usage: 109.3+ KB


,tweet_id,nb_favorites,nb_retweets,nb_user_followers,nb_user_favourites,creation_date
0,892420643555336193,36289,7722,8774885,146020,2017-08-01 16:23:56+00:00
1,892177421306343426,31291,5714,8774885,146020,2017-08-01 00:17:27+00:00
2,891815181378084864,23565,3785,8774885,146020,2017-07-31 00:18:03+00:00
3,891689557279858688,39588,7884,8774885,146020,2017-07-30 15:58:51+00:00
4,891327558926688256,37806,8499,8774885,146020,2017-07-29 16:00:24+00:00


In [43]:
# Control imagePrediction
imagePredictionCleaned.info()
imagePredictionCleaned.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id             2075 non-null object
jpg_url              2075 non-null object
img_num              2075 non-null int64
prediction_1         2075 non-null object
prediction_1_conf    2075 non-null float64
prediction_1_dog     2075 non-null bool
prediction_2         2075 non-null object
prediction_2_conf    2075 non-null float64
prediction_2_dog     2075 non-null bool
prediction_3         2075 non-null object
prediction_3_conf    2075 non-null float64
prediction_3_dog     2075 non-null bool
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 152.1+ KB


,tweet_id,jpg_url,img_num,prediction_1,prediction_1_conf,prediction_1_dog,prediction_2,prediction_2_conf,prediction_2_dog,prediction_3,prediction_3_conf,prediction_3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True


#### Tidyness 1 : 
I had to summarize all 3 predictions group of columns into only 2 columns.
My choice was to maintain only the most confident prediction above the 3 which was detected as being a `dog``

In [44]:
# Let's focus on having a "dog" prediction and confidence for every tweet
predictionColumn = ['prediction_1', 'prediction_2', 'prediction_3']

def getDogPrediction(df, columns, data):
    if (data == 'confidence'):
        suffix = '_conf'
    elif (data == 'prediction'):
        suffix = ''
    else:
        return None
    
    for column in columns: 
        if (df[column + '_dog']==True):
            return df[column + suffix];
    return None

imagePredictionCleaned['prediction'] = imagePredictionCleaned.apply(lambda x: getDogPrediction(x, predictionColumn, 'prediction'), axis=1)
imagePredictionCleaned['confidence'] = imagePredictionCleaned.apply(lambda x: getDogPrediction(x, predictionColumn, 'confidence'), axis=1)

# Dropping column stages
for column in predictionColumn:
    columnToDrop = [column, column + '_dog', column + '_conf']
    imagePredictionCleaned.drop(columns=columnToDrop, inplace=True)


In [45]:
imagePredictionCleaned.head(3)

,tweet_id,jpg_url,img_num,prediction,confidence
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461


#### Tidyness 2 : 
Transforming the `confidence` column which was a percentage to an int value
During this transformation, I'll be removing all tweets where the image prediction did not detect any dog (confidence == NaN)

In [46]:
# Let's transform prediction to an int value (instead of percentage)
imagePredictionCleaned['confidence'] = imagePredictionCleaned['confidence'] * 100

In [48]:
imagePredictionCleaned[imagePredictionCleaned.confidence.isna()]

,tweet_id,jpg_url,img_num,prediction,confidence
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,None,NaN
17,666104133288665088,https://pbs.twimg.com/media/CT56LSZWoAAlJj2.jpg,1,None,NaN
18,666268910803644416,https://pbs.twimg.com/media/CT8QCd1WEAADXws.jpg,1,None,NaN
21,666293911632134144,https://pbs.twimg.com/media/CT8mx7KW4AEQu8N.jpg,1,None,NaN
25,666362758909284353,https://pbs.twimg.com/media/CT9lXGsUcAAyUFt.jpg,1,None,NaN
29,666411507551481857,https://pbs.twimg.com/media/CT-RugiWIAELEaq.jpg,1,None,NaN
45,666786068205871104,https://pbs.twimg.com/media/CUDmZIkWcAAIPPe.jpg,1,None,NaN
50,666837028449972224,https://pbs.twimg.com/media/CUEUva1WsAA2jPb.jpg,1,None,NaN
51,666983947667116034,https://pbs.twimg.com/media/CUGaXDhW4AY9JUH.jpg,1,None,NaN
53,667012601033924608,https://pbs.twimg.com/media/CUG0bC0U8AAw2su.jpg,1,None,NaN


So we have 324 prediction which were not detected as a dog on either the 3 predictions, we must drop those datas because we just have to analyze **dogs** here ;-)

In [49]:
imagePredictionCleaned = imagePredictionCleaned[imagePredictionCleaned.confidence.notnull()]

In [50]:
imagePredictionCleaned.head(4)

,tweet_id,jpg_url,img_num,prediction,confidence
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,46.5074
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,50.6826
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,59.6461
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,40.8143


In [51]:
imagePredictionCleaned['confidence'] = imagePredictionCleaned['confidence'].astype('int')

### Merging all datasets
To be able to work with all those datas, a merge was made using the `tweet_id` column which was the common "joint" column for all 3 datasets.

In [52]:
# Merging all dataset (twitterArchiveCleaned, imagePredictionCleaned, jsonTweetsCleaned) into one df
df = twitterArchiveCleaned.merge(jsonTweetsCleaned, on='tweet_id')
df = df.merge(imagePredictionCleaned, on='tweet_id')
df.info()
df.head(4)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1561 entries, 0 to 1560
Data columns (total 17 columns):
tweet_id              1561 non-null object
timestamp             1561 non-null datetime64[ns, UTC]
source                0 non-null object
text                  1561 non-null object
expanded_urls         1561 non-null object
rating                1561 non-null int64
name                  1561 non-null object
stage                 1561 non-null object
nb_favorites          1561 non-null int64
nb_retweets           1561 non-null int64
nb_user_followers     1561 non-null int64
nb_user_favourites    1561 non-null int64
creation_date         1561 non-null datetime64[ns, UTC]
jpg_url               1561 non-null object
img_num               1561 non-null int64
prediction            1561 non-null object
confidence            1561 non-null int64
dtypes: datetime64[ns, UTC](2), int64(7), object(8)
memory usage: 219.5+ KB


,tweet_id,timestamp,source,text,expanded_urls,rating,name,stage,nb_favorites,nb_retweets,nb_user_followers,nb_user_favourites,creation_date,jpg_url,img_num,prediction,confidence
0,892177421306343426,2017-08-01 00:17:27+00:00,NaN,This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,Tilly,none,31291,5714,8774885,146020,2017-08-01 00:17:27+00:00,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,32
1,891815181378084864,2017-07-31 00:18:03+00:00,NaN,This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12,Archie,none,23565,3785,8774885,146020,2017-07-31 00:18:03+00:00,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,71
2,891689557279858688,2017-07-30 15:58:51+00:00,NaN,This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13,Darla,none,39588,7884,8774885,146020,2017-07-30 15:58:51+00:00,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,Labrador_retriever,16
3,891327558926688256,2017-07-29 16:00:24+00:00,NaN,This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12,Franklin,none,37806,8499,8774885,146020,2017-07-29 16:00:24+00:00,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,55


In [53]:
df.drop_duplicates(inplace=True)
df.shape

(1561, 17)

We still have 1561 Tweets to analyze composed with 17 columns, let's store all those data and proceed with analyze

## Storing

In [54]:
# Saving master dataset into file
df.to_csv('twitter_master_dataset.csv', encoding='utf-8')

## Acting on Wrangled Data

You'll find the second part of the analysis in the ```act_report.pynb``` file